In [1]:
import os
import json
import pandas as pd
import numpy as np
import tqdm
import scipy.sparse as sp
from matplotlib import pyplot as plt
import seaborn as sns
import mglearn
%matplotlib inline
from IPython.display import display
from pprint import pprint
pd.set_option('display.max_columns', 50)
DATA_PATH='D:\download'


In [2]:
t_e = pd.read_csv(
    os.path.join(DATA_PATH, 'transactions_Movregress.csv'),
    
)
t_e['duration']=t_e['duration'].map(lambda x: 5 if x == 0 else x)
t_e.watched_time_y=t_e.watched_time_y/60
t_e.watched_time_x=t_e.watched_time_y/t_e.duration
t_e=t_e.drop([
'duration',
'feature_2',
'feature_3',
'feature_4',
'quantity',
'rating_mean',
'CM_S',
'CM_P',
'CM_R',
'feature_5_65',
'feature_5_00',
'feature_5_68',
'feature_5_59',
'feature_5_-1',
'feature_5_44',
'smallf3',
'middlef3',
'bigf3',
'ts-f1',
'watched_time_y',
'DEV0',
'DEV1',
'DEV2',
'DEV3',
'DEV4',
'DEV5',
'DEV6',
'Unnamed: 0',
'longfilm',
'shortfilm'
],1)
t_e.head()

,element_uid,user_uid,watched_time_x,rating
0,3336,5177,0.792963,9.0
1,3336,349202,0.609444,NaN
2,3336,117582,1.060556,NaN
3,3336,523833,0.804815,NaN
4,3336,571064,0.845926,NaN


In [65]:
ab=t_e

In [66]:
t_e=ab

In [3]:
t_eT=t_e[~t_e.rating.isnull()]
t_eF=t_e[t_e.rating.isnull()]
t_eF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8098440 entries, 1 to 8447927
Data columns (total 4 columns):
element_uid       int64
user_uid          int64
watched_time_x    float64
rating            float64
dtypes: float64(2), int64(2)
memory usage: 308.9 MB


In [4]:
t_eT.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 349488 entries, 0 to 8447904
Data columns (total 4 columns):
element_uid       349488 non-null int64
user_uid          349488 non-null int64
watched_time_x    349488 non-null float64
rating            349488 non-null float64
dtypes: float64(2), int64(2)
memory usage: 13.3 MB


In [6]:
t_eF=t_eF[t_eF.watched_time_x >0.3]
t_eF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5731986 entries, 1 to 8447927
Data columns (total 4 columns):
element_uid       int64
user_uid          int64
watched_time_x    float64
rating            float64
dtypes: float64(2), int64(2)
memory usage: 218.7 MB


In [5]:
from collections import Counter
Counter(t_eT.rating)

Counter({9.0: 47954,
         8.0: 76113,
         10.0: 126548,
         7.0: 29825,
         6.0: 28504,
         5.0: 13509,
         3.0: 3994,
         2.0: 7132,
         4.0: 14892,
         0.0: 227,
         1.0: 790})

In [7]:
t_eT=t_eT[t_eT.rating >3]
t_eT1=t_eT[t_eT.watched_time_x >0.4]
t_eT2=t_eT[t_eT.watched_time_x <= 0.4]    
t_eT.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 337345 entries, 0 to 8447904
Data columns (total 4 columns):
element_uid       337345 non-null int64
user_uid          337345 non-null int64
watched_time_x    337345 non-null float64
rating            337345 non-null float64
dtypes: float64(2), int64(2)
memory usage: 12.9 MB


In [10]:

t_eT1['interaction']=t_eT1['rating']
t_eT2['interaction']=t_eT2['watched_time_x']*10
t_eT2['interaction']=t_eT2['interaction'].map(lambda x: 10 if x> 10 else x)

t_eF['interaction']=t_eF['watched_time_x']*10
t_eF['interaction']=t_eF['interaction'].map(lambda x: 10 if x> 10 else x)
t_eF=t_eF.drop(['watched_time_x', 'rating'],1)
t_eT1=t_eT1.drop(['watched_time_x', 'rating'],1)
t_eT2=t_eT2.drop(['watched_time_x', 'rating'],1)
t_e=t_eT1.append(t_eF)
t_e=t_e.append(t_eT2)
t_e.info()


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pyda

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6069331 entries, 0 to 8447284
Data columns (total 3 columns):
element_uid    int64
user_uid       int64
interaction    float64
dtypes: float64(1), int64(2)
memory usage: 185.2 MB


In [11]:
t_e['rating']=t_e['interaction']
t_e=t_e.drop(['interaction'],1)
t_e.head()

,element_uid,user_uid,rating
0,3336,5177,9.0
43,3336,86893,8.0
57,3336,419459,10.0
67,3336,184988,7.0
128,3336,521182,8.0


In [12]:

t_e.to_csv("WTforfilms(evr0204).csv")

In [13]:
t_e.isnull().sum()/t_e.shape[0]

element_uid    0.0
user_uid       0.0
rating         0.0
dtype: float64

In [67]:
t_e[t_e.rating.isnull()]

,element_uid,user_uid,rating
2930905,231,98325,NaN
2930945,231,293038,NaN
2931069,231,584825,NaN
2931125,231,293304,NaN
2931148,231,74775,NaN
2931161,231,353209,NaN
2931267,231,415651,NaN
2931311,231,62930,NaN
2931324,231,284705,NaN
5510350,1043,499649,NaN


In [10]:
np.isinf(t_e).sum()

element_uid       0
user_uid          0
watched_time_x    0
rating            0
duration          0
watched_time_y    0
dtype: int64

In [2]:
t_e = pd.read_csv(
    os.path.join(DATA_PATH, 'transactions_series_toregress.csv'),
    
)
t_e=t_e.drop([
'minus',
'feature_2',
'feature_3',
'feature_4',
'quantity',
'rating_mean',
'CM_S',
'CM_P',
'CM_R',
'feature_5_65',
'feature_5_00',
'feature_5_68',
'feature_5_59',
'feature_5_-1',
'feature_5_44',
'smallf3',
'middlef3',
'bigf3',
'ts-f1',
'DEV0',
'DEV1',
'DEV3',
'DEV4',
'DEV5',
'DEV6',
'Unnamed: 0',
],1)
t_e['duration']=t_e['duration'].map(lambda x: 5 if x == 0 else x)
t_e.watched_time_y=t_e.watched_time_y/60
t_e.watched_time_x=t_e.watched_time_y/t_e.duration
t_e.head()

,element_uid,user_uid,watched_time_x,rating,duration,watched_time_y
0,5651,490059,7.825000,NaN,20,156.500000
1,5651,278741,1.166667,NaN,20,23.333333
2,5651,469687,5.044167,NaN,20,100.883333
3,5651,371512,1.880000,NaN,20,37.600000
4,5651,566802,2.121667,NaN,20,42.433333


In [49]:

t_e['porog']=t_e['duration']
dct0=  {20 : 24, 40: 10,  60: 8, 50: 10, 5:24 , 30:20, 80: 6, 10: 24, 70: 5, 90:4, 120:3,  100:3}
t_e.porog=t_e.porog.map(dct0)
t_e.porog=t_e.watched_time_x/t_e.porog

In [54]:
t_eT=t_e[~t_e.rating.isnull()]
t_eF=t_e[t_e.rating.isnull()]
t_eT['interaction']=t_eT.rating
t_eF['interaction']=t_eF.porog*10
t_eF['interaction']=t_eF['interaction'].map(lambda x: 10 if x>10 else x)
t_eF=t_eF.drop(['watched_time_x', 'rating', 'duration','watched_time_y','porog'],1)
t_eT=t_eT.drop(['watched_time_x', 'rating', 'duration','watched_time_y','porog'],1)
t_e=t_eT.append(t_eF)
t_e['rating']=t_e['interaction']
t_e=t_e.drop(['interaction'],1)
t_e.head()

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

,element_uid,user_uid,rating
396,5651,168651,9.0
805,5651,166597,10.0
851,5651,415491,9.0
1061,5651,119545,10.0
2510,5651,180699,10.0


In [3]:
t_eT=t_e[~t_e.rating.isnull()]
t_eF=t_e[t_e.rating.isnull()]
t_eT['interaction']=t_eT.rating
t_eF['interaction']=t_eF.watched_time_x * 2
t_eF['interaction']=t_eF['interaction'].map(lambda x: 10 if x>10 else x)
t_eF=t_eF.drop(['watched_time_x', 'rating', 'duration','watched_time_y'],1)
t_eT=t_eT.drop(['watched_time_x', 'rating', 'duration','watched_time_y'],1)
t_e=t_eT.append(t_eF)
t_e['rating']=t_e['interaction']
t_e=t_e.drop(['interaction'],1)
t_e.head()

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

,element_uid,user_uid,rating
396,5651,168651,9.0
805,5651,166597,10.0
851,5651,415491,9.0
1061,5651,119545,10.0
2510,5651,180699,10.0


In [4]:

t_e.to_csv("WTforserials(num3).csv")

In [6]:
from collections import Counter

In [43]:
from collections import Counter
len(pd.unique(t_e.element_uid))

823

In [15]:
t_e.watched_time_x=t_e.watched_time_x.fillna(100)

In [55]:
max(t_e[t_e.duration == 120].watched_time_y)/120

4.648750000000001

In [56]:
round(max(t_e[t_e.duration == 120].watched_time_y)/120)

5

In [58]:
from math import *
floor(max(t_e[t_e.duration == 120].watched_time_y)/120)

4

In [49]:
Counter(t_e.duration)

Counter({20: 163249,
         40: 240151,
         60: 104792,
         50: 347677,
         5: 99028,
         30: 54761,
         80: 2528,
         10: 61308,
         70: 1146,
         90: 854,
         120: 88,
         100: 570})

In [16]:
t_e['quas']=t_e.element_uid.map(t_e.groupby("element_uid").watched_time_y.max())
t_e['quas']=np.floor(t_e['quas']/t_e.duration)
t_e.quas=t_e.quas.fillna(5)
t_e['quas']=t_e.watched_time_x/t_e['quas']

In [18]:
t_e.head()

,element_uid,user_uid,watched_time_x,rating,duration,watched_time_y,quas
0,5651,490059,7.825000,NaN,20,156.500000,0.019612
1,5651,278741,1.166667,NaN,20,23.333333,0.002924
2,5651,469687,5.044167,NaN,20,100.883333,0.012642
3,5651,371512,1.880000,NaN,20,37.600000,0.004712
4,5651,566802,2.121667,NaN,20,42.433333,0.005317


In [27]:
t_e.quas=t_e.quas.fillna(0)
t_e.quas[t_e.quas == np.inf]=0

D:\Anaconda3\lib\site-packages\pandas\core\generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [35]:
t_eT=t_e[~t_e.rating.isnull()]
t_eF=t_e[t_e.rating.isnull()]

In [36]:

t_eT['interaction']=t_eT.rating
t_eT.head()

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,element_uid,user_uid,watched_time_x,rating,duration,watched_time_y,quas,interaction
396,5651,168651,15.320000,9.0,20,306.400000,0.038396,9.0
805,5651,166597,8.374167,10.0,20,167.483333,0.020988,10.0
851,5651,415491,12.863333,9.0,20,257.266667,0.032239,9.0
1061,5651,119545,16.375000,10.0,20,327.500000,0.041040,10.0
2510,5651,180699,16.741667,10.0,20,334.833333,0.041959,10.0


In [37]:

t_eF['interaction']=t_eF.quas*10
t_eF['interaction']=t_eF['interaction'].map(lambda x: 10 if x>3 else x)

t_eF['interaction']=t_eF['interaction'].map(lambda x: 5 if x>2 and x <3 else x)
t_eF['interaction']=t_eF['interaction'].map(lambda x: x*2 if x <2 else x)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexi

In [38]:
t_eF.head()

,element_uid,user_uid,watched_time_x,rating,duration,watched_time_y,quas,interaction
0,5651,490059,7.825000,NaN,20,156.500000,0.019612,0.392231
1,5651,278741,1.166667,NaN,20,23.333333,0.002924,0.058480
2,5651,469687,5.044167,NaN,20,100.883333,0.012642,0.252840
3,5651,371512,1.880000,NaN,20,37.600000,0.004712,0.094236
4,5651,566802,2.121667,NaN,20,42.433333,0.005317,0.106349


In [39]:
t_eF=t_eF.drop(['watched_time_x', 'rating', 'duration','watched_time_y','quas'],1)
t_eT=t_eT.drop(['watched_time_x', 'rating', 'duration','watched_time_y','quas'],1)
t_e=t_eT.append(t_eF)
t_e['rating']=t_e['interaction']
t_e=t_e.drop(['interaction'],1)
t_e.head()

,element_uid,user_uid,rating
396,5651,168651,9.0
805,5651,166597,10.0
851,5651,415491,9.0
1061,5651,119545,10.0
2510,5651,180699,10.0


In [40]:
t_e.to_csv("WTforserials.csv")

In [2]:
t_e = pd.read_csv(
    os.path.join(DATA_PATH, 'transactions_MMtoregress.csv'),
    
)
t_e=t_e.drop(['feature_2',
'Unnamed: 0',              
'feature_3',
'feature_4',
'quantity',
'rating_mean',
'CM_S',
'CM_P',
'CM_R',
'feature_5_65',
'feature_5_00',
'feature_5_68',
'feature_5_59',
'feature_5_-1',
'feature_5_44',
'smallf3',
'middlef3',
'bigf3',
'ts-f1',
'DEV0',
'DEV1',
'DEV3',
'DEV4',
'DEV5',
'DEV6'],1)
t_e['duration']=t_e['duration'].map(lambda x: 5 if x == 0 else x)
t_e.watched_time_y=t_e.watched_time_y/60
t_e.watched_time_x=t_e.watched_time_y/t_e.duration
t_e.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118932 entries, 0 to 118931
Data columns (total 6 columns):
element_uid       118932 non-null int64
user_uid          118932 non-null int64
watched_time_x    118932 non-null float64
rating            462 non-null float64
duration          118932 non-null int64
watched_time_y    118932 non-null float64
dtypes: float64(3), int64(3)
memory usage: 5.4 MB


In [3]:
t_e.head()

,element_uid,user_uid,watched_time_x,rating,duration,watched_time_y
0,7347,182294,3.562333,NaN,50,178.116667
1,7347,121110,3.591333,NaN,50,179.566667
2,7347,371740,1.924000,NaN,50,96.200000
3,7347,476818,1.120000,NaN,50,56.000000
4,7347,348183,1.519667,NaN,50,75.983333


In [4]:
len(pd.unique(t_e.element_uid))

172

In [15]:
t_eT.interaction

67        10.0
500       10.0
518       10.0
537       10.0
877       10.0
1074       3.0
1339       9.0
1433      10.0
1545       7.0
1633      10.0
1711       9.0
1939       9.0
2015      10.0
2018      10.0
2036      10.0
2193       9.0
2239      10.0
2282       6.0
2300       9.0
2830       8.0
6163      10.0
6687      10.0
6768      10.0
6975      10.0
7090      10.0
7275      10.0
9213      10.0
9899      10.0
10438     10.0
10572     10.0
          ... 
115259     8.0
115519     7.0
115674    10.0
115750     9.0
115903     8.0
116046     8.0
116081    10.0
116229     9.0
116653    10.0
116713    10.0
116810     9.0
116819     8.0
116860    10.0
117011     9.0
117162     8.0
117183    10.0
117201     8.0
117458    10.0
117476     5.0
117648     8.0
117733    10.0
117737     9.0
118038    10.0
118086    10.0
118166     9.0
118189    10.0
118190    10.0
118553     7.0
118657    10.0
118813     8.0
Name: interaction, Length: 462, dtype: float64

In [13]:
t_eT=t_e[~t_e.rating.isnull()]
t_eF=t_e[t_e.rating.isnull()]
t_eT['interaction']=t_eT.rating
t_eF['interaction']=t_eF.watched_time_x * 5
t_eF['interaction']=t_eF['interaction'].map(lambda x: 10 if x>10 else x)
t_eF=t_eF.drop(['watched_time_x', 'rating', 'duration','watched_time_y'],1)
t_eT=t_eT.drop(['watched_time_x', 'rating', 'duration','watched_time_y'],1)
t_e=t_eT.append(t_eF)
t_e['rating']=t_e['interaction']
t_e=t_e.drop(['interaction'],1)
t_e.head()

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

,element_uid,user_uid,rating
67,7347,570139,10.0
500,7347,490238,10.0
518,7347,473312,10.0
537,7347,29012,10.0
877,7347,236470,10.0


In [16]:
t_e.to_csv("WTformnmov.csv")